# Q3 — Impact Analysis (主线)

目的：回答“哪些因素影响技术线（评委）与人气线（粉丝）”。

对应主线代码：
- `src/mcm2026/pipelines/mcm2026c_q3_mixed_effects_impacts.py`
- `src/mcm2026/visualizations/q3_visualizations.py`

主要输出：
- `outputs/tables/mcm2026c_q3_impact_analysis_coeffs.csv`

不确定性处理：
- 对粉丝线（fan line）会进行 `n_refits` 次重抽样（使用 Q1 给出的 p05/p95 估计的 sd）
- 输出使用 5%/95% 分位数作为区间（median + [p05,p95]）


## 先用一句话说清楚 Q3（口语版）

Q3 的目标不是“预测冠军”，而是回答：
> **哪些因素影响“技术线”（评委更喜欢谁）与“人气线”（观众更支持谁）？**

我们把系统拆成两条线：
- **技术线**：`judge_score_pct_mean`（评委百分位的赛季平均）
- **人气线**：`fan_vote_index_mean`（由 Q1 反推得到的观众支持强度）

### Q3 的不确定性是怎么来的？
人气线来自 Q1 推断，本身就有区间。
所以 Q3 在拟合“人气线”时，会做 `n_refits` 次“带噪声重抽样”，最后汇总成一个系数区间。

论文手可以这样理解：
> 我们不是只拟合一次，而是把 Q1 的不确定性传播到 Q3，得到更稳健的影响因素结论。

In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import pandas as pd

def _find_repo_root(start: Path | None = None) -> Path:
    here = Path.cwd() if start is None else Path(start)
    for p in [here, *here.parents]:
        if (p / 'pyproject.toml').exists():
            return p
    return here

REPO_ROOT = _find_repo_root()
SRC_DIR = REPO_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

REPO_ROOT


In [ ]:
# 工具函数：在 notebook 里自动展示 outputs/figures 下的图片（优先 TIFF，其次提示 EPS 路径）
from pathlib import Path


def show_saved_figures(question: str, stems: list[str]) -> None:
    """Display figures saved by visualization scripts.

    - Prefer TIFF (good for notebook preview)
    - EPS usually cannot be previewed in Jupyter without额外系统依赖，所以这里只打印路径
    """

    try:
        from IPython.display import Image, display
    except Exception:
        print('⚠️ IPython not available; cannot display images inline.')
        return

    q = question.strip().lower()
    tiff_dir = Path(REPO_ROOT) / 'outputs' / 'figures' / q / 'tiff'
    eps_dir = Path(REPO_ROOT) / 'outputs' / 'figures' / q / 'eps'

    shown = 0
    for stem in stems:
        tiff_fp = tiff_dir / f'{stem}.tiff'
        eps_fp = eps_dir / f'{stem}.eps'

        if tiff_fp.exists():
            display(Image(filename=str(tiff_fp)))
            shown += 1
        elif eps_fp.exists():
            print('EPS figure exists (not previewed inline):', eps_fp)

    if shown == 0:
        print('⚠️ No TIFF figures found under:', tiff_dir)


# 用法示例（Q3）：
# show_saved_figures('q3', ['q3_judge_vs_fan_forest_plot'])

## 一键运行（推荐给新队友/论文手）

运行下面这一格会：
- 自动跑 Q0 → Q1 → Q3
- 输出系数表（含区间）
- 生成 Q3 图（TIFF/EPS）
- 并把关键 TIFF 图直接显示在 notebook 里

如果你只想“快速得到可写进论文的图”，跑这一格即可。

In [ ]:
import traceback

try:
    import sys
    from pathlib import Path

    import pandas as pd

    # ---- Make this cell self-contained (no dependency on earlier cells) ----
    def _find_repo_root(start: Path | None = None) -> Path:
        here = Path.cwd() if start is None else Path(start)
        for p in [here, *here.parents]:
            if (p / 'pyproject.toml').exists():
                return p
        return here

    if 'REPO_ROOT' not in globals() or not isinstance(globals().get('REPO_ROOT'), Path):
        REPO_ROOT = _find_repo_root()

    SRC_DIR = REPO_ROOT / 'src'
    if str(SRC_DIR) not in sys.path:
        sys.path.insert(0, str(SRC_DIR))

    try:
        from IPython.display import display
    except Exception:
        def display(x):
            return x

    from mcm2026.pipelines import (
        mcm2026c_q0_build_weekly_panel,
        mcm2026c_q1_smc_fan_vote,
        mcm2026c_q3_mixed_effects_impacts,
    )
    from mcm2026.visualizations.config import VisualizationConfig, create_output_directories
    from mcm2026.visualizations.q3_visualizations import generate_all_q3_visualizations

    _ = mcm2026c_q0_build_weekly_panel.run()
    _ = mcm2026c_q1_smc_fan_vote.run()

    q3_out = mcm2026c_q3_mixed_effects_impacts.run()
    print('✅ Q3 done:', q3_out)

    coeffs = pd.read_csv(q3_out.impact_coeffs_csv)
    print('rows:', len(coeffs))
    display(coeffs.head(10))

    viz_cfg = VisualizationConfig()
    out_dirs = create_output_directories(REPO_ROOT / 'outputs' / 'figures', ['Q3'])
    generate_all_q3_visualizations(REPO_ROOT, out_dirs['Q3'], viz_cfg, showcase=False)

    # Show TIFFs (best-effort)
    try:
        from IPython.display import Image, display as _display

        tiff_dir = REPO_ROOT / 'outputs' / 'figures' / 'q3' / 'tiff'
        candidates = [
            tiff_dir / 'q3_judge_vs_fan_forest_plot.tiff',
            tiff_dir / 'q3_effect_size_comparison.tiff',
            tiff_dir / 'q3_industry_impact_heatmap.tiff',
        ]

        shown = 0
        for fp in candidates:
            if fp.exists():
                _display(Image(filename=str(fp)))
                shown += 1
        if shown == 0:
            print('⚠️ No TIFF figures found to display yet. They should be under:', tiff_dir)
    except Exception:
        pass

except Exception as e:
    print('❌ One-click Q3 failed:', e)
    traceback.print_exc()

## 1) 确保 Q0 + Q1 已运行（必要依赖）

Q3 依赖：
- Q0：weekly_panel + season_features
- Q1：posterior summary（用于 fan line）


In [ ]:
from mcm2026.pipelines import mcm2026c_q0_build_weekly_panel, mcm2026c_q1_smc_fan_vote

_ = mcm2026c_q0_build_weekly_panel.run()
_ = mcm2026c_q1_smc_fan_vote.run()
'ok'


## 2) 运行 Q3

输出表 `mcm2026c_q3_impact_analysis_coeffs.csv` 的核心列：
- `outcome`：judge_score_pct_mean（技术线） / fan_vote_index_mean（人气线）
- `estimate`、`ci_low`、`ci_high`：系数点估计与区间

注意：fan line 的区间来自 `n_refits` 次重采样聚合，不是单次回归的标准误。

In [ ]:
from mcm2026.pipelines import mcm2026c_q3_mixed_effects_impacts

q3_out = mcm2026c_q3_mixed_effects_impacts.run()
q3_out


In [ ]:
coeffs = pd.read_csv(q3_out.impact_coeffs_csv)
coeffs.head(10)


## 常见问答（FAQ）

### Q3 为什么不像传统论文那样“只给一堆 p-value”？
因为我们的人气线（fan line）来自 Q1 推断，有不确定性。

如果我们假装 Q1 是“精确观测值”，再去做 p-value，会显得非常不诚实。

所以我们采用的表达更像：
- 给一个“中心估计”（median / estimate）
- 再给一个“传播后的区间”（ci_low/ci_high）

### `ci_low/ci_high` 是怎么来的？
对 fan line：我们做了 `n_refits` 次重抽样（把 Q1 的不确定性注入进去），最后取分位数。

论文口径：
> 我们把 Q1 的不确定性传播到 Q3，从而得到更稳健的影响因素结论。

### Mixed effects 有什么用？
直觉：DWTS 有很多“分组效应”，比如不同职业/不同舞伴体系可能有系统差异。
Mixed model 能更自然地吸收这类结构性差异。

但如果 MixedLM 不收敛，代码会回退到 OLS，保证主线可跑、输出可解释。

## 3) 口语化解释：怎么读这张“系数表”？

你可以把每一行理解成：
> “某个因素每增加一点点，技术线/人气线会往哪个方向变化？”

关键列：
- `estimate`：影响方向与强度（正=增加，负=降低）
- `ci_low` / `ci_high`：我们做了不确定性传播后的一个区间（人气线尤其重要）

下面我们直接把几个最常见的解释变量（如年龄、是否美国、活跃周数等）画成一张“森林图”，
让非技术队友也能一眼看懂：
- 技术线（评委）偏好什么
- 人气线（粉丝）偏好什么

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 简单做一个“森林图”：把常见变量的系数和区间画出来

def _pick_terms(df: pd.DataFrame) -> list[str]:
    preferred = ['age', 'age_sq', 'is_us', 'log_state_pop', 'n_weeks_active']
    terms = [t for t in preferred if t in set(df['term'].astype(str))]

    # 再补几个行业项（如果有）
    ind = [t for t in df['term'].astype(str).unique() if t.startswith('C(industry)')]
    terms += ind[:3]
    return terms


def _forest(ax, df: pd.DataFrame, title: str):
    d = df.copy()
    d['term'] = d['term'].astype(str)
    d = d[~d['term'].str.contains('Intercept', case=False, na=False)].copy()

    terms = _pick_terms(d)
    if not terms:
        ax.text(0.5, 0.5, 'No expected terms found', ha='center', va='center')
        ax.axis('off')
        return

    d = d[d['term'].isin(terms)].copy()
    d['estimate'] = pd.to_numeric(d['estimate'], errors='coerce')
    d['ci_low'] = pd.to_numeric(d['ci_low'], errors='coerce')
    d['ci_high'] = pd.to_numeric(d['ci_high'], errors='coerce')
    d = d.dropna(subset=['estimate', 'ci_low', 'ci_high']).copy()

    d = d.sort_values('estimate')
    y = np.arange(len(d))
    ax.errorbar(
        d['estimate'],
        y,
        xerr=[d['estimate'] - d['ci_low'], d['ci_high'] - d['estimate']],
        fmt='o',
        capsize=3,
        color='black',
        ecolor='gray',
        alpha=0.85,
    )
    ax.axvline(0.0, color='red', linestyle='--', alpha=0.6)
    ax.set_yticks(y)
    ax.set_yticklabels([t.replace('C(industry)[T.', '').replace(']', '') for t in d['term']])
    ax.set_title(title)
    ax.set_xlabel('Coefficient (with interval)')
    ax.grid(True, alpha=0.3)


fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=False)

j = coeffs[coeffs['outcome'].astype(str) == 'judge_score_pct_mean'].copy()
f = coeffs[coeffs['outcome'].astype(str) == 'fan_vote_index_mean'].copy()

_forest(axes[0], j, 'Technical line (judges)')
_forest(axes[1], f, 'Popularity line (fans)')

plt.tight_layout()
plt.show()

## 3) 生成 Q3 图

图的输出目录：`outputs/figures/Q3/`（TIFF + EPS）。

In [ ]:
from mcm2026.visualizations.config import VisualizationConfig, create_output_directories
from mcm2026.visualizations.q3_visualizations import generate_all_q3_visualizations

viz_cfg = VisualizationConfig()
out_dirs = create_output_directories(REPO_ROOT / 'outputs' / 'figures', ['Q3'])
generate_all_q3_visualizations(REPO_ROOT, out_dirs['Q3'], viz_cfg, showcase=False)
out_dirs['Q3']
